In [38]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime

from tqdm.notebook import tqdm

%matplotlib notebook

In [26]:
exp_dir = "/Users/romainegele/Documents/Argonne/deephyper-scalable-bo/experiments/polaris/dhb/output/"

exp_name = "dhb_combo-OPT-TPE-SHA-40-10800-42"
# exp_name = "dhb_combo-DBO-RF-UCB-SHA-40-10800-42"

path = os.path.join(exp_dir, exp_name, "results.csv")

df = pd.read_csv(path, index_col=0)

/Users/romainegele/miniforge3/envs/dh-arm/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (23,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
df

,p:activation,p:base_lr,p:batch_normalization,p:batch_size,p:dense_0,p:dense_1,p:dense_2,p:dense_feature_layers_0,p:dense_feature_layers_1,p:dense_feature_layers_2,...,m:train_r2,m:train_corr,m:valid_mse,m:valid_mae,m:valid_r2,m:valid_corr,m:test_mse,m:test_mae,m:test_r2,m:test_corr
0,hard_sigmoid,0.000428,False,33,193,251,54,78,10,80,...,-1.971944,0.000375,0.821187,0.747404,-1.982072,-0.004594,0.824265,0.750007,-1.978787,0.000644
1,tanh,0.000153,True,18,167,161,56,542,142,110,...,0.441521,0.665335,0.153919,0.301202,0.441056,0.665146,0.154070,0.301387,0.443211,0.666508
2,elu,0.002575,True,75,354,692,22,11,496,110,...,-0.389251,0.030035,0.383793,0.534991,-0.393711,0.030062,0.385680,0.536282,-0.393796,0.028001
3,swish,0.000016,True,464,12,11,47,98,72,68,...,-0.435131,0.000717,0.395231,0.538443,-0.435248,0.004665,0.398651,0.541447,-0.440671,-0.006088
4,hard_sigmoid,0.002488,False,26,108,322,245,150,116,71,...,0.587538,0.772514,0.113620,0.244130,0.587397,0.772951,0.114857,0.245135,0.584922,0.770748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18862,gelu,0.000170,False,216,31,608,228,536,440,160,...,0.581528,0.771363,0.115380,0.235537,0.581008,0.771306,0.117320,0.237203,0.576021,0.768164
18863,gelu,0.000165,False,411,539,613,273,66,581,158,...,0.795577,0.901832,0.060471,0.164473,0.780404,0.894192,0.060250,0.163556,0.782264,0.894896
18864,gelu,0.000085,False,378,91,620,275,168,10,876,...,0.339849,0.615202,0.181417,0.305542,0.341199,0.615338,0.184478,0.306098,0.333321,0.611709
18865,gelu,0.000055,False,305,30,615,11,536,443,35,...,0.614192,0.786736,0.107056,0.228599,0.611234,0.785174,0.107489,0.229076,0.611550,0.785004


In [28]:
hp_names = [cname for cname in df.columns if "p:" in cname]
hp_names

['p:activation',
 'p:base_lr',
 'p:batch_normalization',
 'p:batch_size',
 'p:dense_0',
 'p:dense_1',
 'p:dense_2',
 'p:dense_feature_layers_0',
 'p:dense_feature_layers_1',
 'p:dense_feature_layers_2',
 'p:dropout',
 'p:early_stopping',
 'p:early_stopping_patience',
 'p:learning_rate',
 'p:loss',
 'p:optimizer',
 'p:reduce_lr',
 'p:reduce_lr_factor',
 'p:reduce_lr_patience',
 'p:residual',
 'p:scaling',
 'p:warmup_lr']

In [29]:
hp_types = [df[k].dtype for k in hp_names]
hp_types

[dtype('O'),
 dtype('float64'),
 dtype('bool'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('bool'),
 dtype('int64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('float64'),
 dtype('int64'),
 dtype('bool'),
 dtype('O'),
 dtype('bool')]

In [30]:
df_str = df.astype({k:str for k in hp_names})
df_str["hash"] = df_str[hp_names].agg('-'.join, axis=1)
df_str = df_str[["hash", "objective", "m:timestamp_end"]]
df_str

,hash,objective,m:timestamp_end
0,hard_sigmoid-0.0004279402454597-False-33-193-2...,-1,1.686507e+09
1,tanh-0.0001529959698619-True-18-167-161-56-542...,0.4410562278231549,1.686507e+09
2,elu-0.0025754036957428-True-75-354-692-22-11-4...,-0.3937109109425687,1.686507e+09
3,swish-1.5862073566183983e-05-True-464-12-11-47...,-0.43524777752829036,1.686507e+09
4,hard_sigmoid-0.0024878425075417-False-26-108-3...,0.5873971776287898,1.686507e+09
...,...,...,...
18862,gelu-0.0001700930598376-False-216-31-608-228-5...,0.581008,1.686517e+09
18863,gelu-0.000165294762875-False-411-539-613-273-6...,0.780404,1.686517e+09
18864,gelu-8.54206514216063e-05-False-378-91-620-275...,0.341199,1.686517e+09
18865,gelu-5.524741614760587e-05-False-305-30-615-11...,0.611234,1.686517e+09


In [36]:
df_str.objective.astype(str).str.startswith("F").astype(int).sum()

21

In [37]:
num_evals = len(df_str)

df_str = df_str[~df_str.objective.astype(str).str.startswith("F")]
num_evals_without_failures = len(df_str)

df_str.objective = df_str.objective.astype(float)

num_evals_without_duplicates = len(df_str["hash"].drop_duplicates())

print(f"#eval:{num_evals} - #eval no F:{num_evals_without_failures} - #unique-evals:{num_evals_without_duplicates}")

#eval:18867 - #eval no F:18846 - #unique-evals:18846


/Users/romainegele/miniforge3/envs/dh-arm/lib/python3.10/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
id_max = df_str["objective"].argmax()
hash_max = df_str.iloc[id_max]["hash"]
objective_max = df_str.iloc[id_max]["objective"]

TypeError: reduction operation 'argmax' not allowed for this dtype

In [9]:
best_config_repetitions = df_str[df_str["hash"] == hash_max]
num_occurence = len(best_config_repetitions)
min_objective = best_config_repetitions["objective"].min()
max_objective = best_config_repetitions["objective"].max()
print(f"#occ:{num_occurence} - min:{min_objective:.3f} - max:{max_objective:.3f}")

#occ:1 - min:0.920 - max:0.920


In [10]:
earliest_timestamps = (df_str.groupby("hash")
                       .agg({"timestamp_end": "min"})
                       .reset_index()["timestamp_end"]
                       .values)
all_timestamps = df_str["timestamp_end"].values
indices = np.where(np.in1d(earliest_timestamps, all_timestamps))[0]

In [11]:
df_str.loc[:, ("earliest-occurence",)] = 0
df_str.loc[indices, ("earliest-occurence",)] = 1
df_str = df_str.sort_values("timestamp_end")
df_str = df_str.reset_index(drop=True)

In [12]:
df_str.loc[:, ("better-than-baseline",)] = 0
df_str.loc[(df_str["objective"] >= 0.88), ("better-than-baseline",)] = 1
df_str.loc[:, ("count",)] = (df_str["earliest-occurence"] & df_str["better-than-baseline"]).astype(int)
df_str.loc[:, ("count",)] = df_str["count"].cumsum()
df_str

,hash,objective,timestamp_end,earliest-occurence,better-than-baseline,count
0,hard_sigmoid-0.0069570787597008-False-283-637-...,-0.073699,29.810861,1,0,0
1,selu-0.0004566767026172-True-206-132-138-281-4...,-0.831534,30.178641,1,0,0
2,linear-1.976428084903253e-05-True-310-116-91-4...,-1.000000,30.423589,1,0,0
3,sigmoid-0.0010906038716589-True-462-501-260-24...,-1.000000,30.512007,1,0,0
4,swish-0.0003894922591863-True-354-131-848-28-6...,-0.027499,31.088678,1,0,0
...,...,...,...,...,...,...
47173,swish-0.0001495123090435-True-157-960-164-13-3...,0.623651,10545.735687,1,0,3386
47174,softsign-0.0011695701694482-False-276-151-913-...,-1.000000,10545.756016,1,0,3386
47175,swish-2.3830980948506262e-05-True-301-360-643-...,0.679460,10546.840499,1,0,3386
47176,relu-3.072028252773362e-05-True-210-417-123-14...,0.593661,10547.265661,1,0,3386
